In [0]:
import re
import os
import cv2
import imageio
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random

from keras.layers import Input, Conv2D, Lambda, Dense, Flatten, Dropout, MaxPooling2D, concatenate
from keras.models import Model, Sequential, load_model
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
from tqdm import tnrange

In [0]:
def loadimgs(path):
  X = {}

  max_count = 100
  for filename in os.listdir(path):          
    m = re.search("gallery(\d+)\.jpg", filename)
    if not m:
      continue          
    num = m.groups()[0]

    file_path = os.path.join(path, filename)
    img = cv2.imread(file_path)

# cv2 changes channels order :/
    img = np.flip(img, 2)

    X[int(num)] = img

    if len(X) >= max_count:
      break

  print(len(X))
  return X

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
DATA_BASE_DIR = '/content/gdrive/My Drive/proctf_service_gallery2'

In [0]:
model = load_model(os.path.join(DATA_BASE_DIR, 'model_big_compact_256_2_30_30_30_30_predict.h5'))

for layer in model.layers:
  layer.trainable = False
  
model.summary()

In [0]:
X = loadimgs(os.path.join(DATA_BASE_DIR, 'exploit_imgs'))

In [0]:
sess = K.get_session()

In [0]:
def hack_embs(embs, dists):
  dims = len(embs) - 1
  a = embs[:dims]-embs[dims]
  b = np.array([(np.dot(e,e) - np.dot(embs[dims],embs[dims]) -d*d + dists[dims]*dists[dims])/2 for e,d in zip(embs[:dims], dists[:dims])])

  return np.linalg.solve(a, b)


def generate_dists(target_img):
  target_emb = model.predict(np.array([target_img]))[0].astype(np.float64)
  
  count = len(target_emb) + 1
  random_imgs = []
  for i in range(count):
    random_imgs.append(np.random.random_sample(target_img.shape).astype(np.float32))
  
  random_embs = np.array([model.predict(np.array([random_imgs[n]]))[0] for n in range(0,count)]).astype(np.float64) 
  dists = np.array([np.linalg.norm(e - target_emb) for e in random_embs])
  return (target_emb, random_embs, dists)




In [0]:
def calc_real_loss(sess, model, target_embed, generated_img):
  return np.linalg.norm(target_embed - model.predict(generated_img.eval(sess).astype(np.uint8)))

def create_initial_img(model, target_embed, img_shape, preview):
# Sploit 1
# Initializing with zoomed preview
    return cv2.resize(preview, dsize=(256, 256), interpolation=cv2.INTER_CUBIC).reshape(img_shape)
  
# Sploit 2
# Initializing with constant colors
#################
#   red = np.zeros(img_shape)
#   red[:, :, :, 0] = 128
#   green = np.zeros(img_shape)
#   green[:, :, :, 1] = 128
#   blue = np.zeros(img_shape)
#   blue[:, :, :, 2] = 128
#   gray = np.ones(img_shape) * 128
#   images = [ red, green, blue, gray ]
#      
#   return min(images, key=lambda img: np.linalg.norm(target_embed - model.predict(img)))

# Sploit 3
# Initliazing with grey
##################
# return np.ones(img_shape) * 128

# Sploit 4
# Initializing with a close image
##################
#   random_imgs = [
#       random.choice(X).reshape(img_shape)
#       for i in range(100)
#   ]
#   random_imgs = [
#       img for img in random_imgs
#       if not np.all(np.isclose(target_embed, model.predict(img)))
#   ]
#   return min(random_imgs, key=lambda img: np.linalg.norm(target_embed - model.predict(img)))



def generate_reverse_image(sess, model, target_embed, real_emb, img_shape, preview, lr=0.1, max_steps=1000, stop_loss=0.05):
  target_embed_tensor = tf.constant(target_embed)
  generated_img = tf.Variable(np.zeros(img_shape), trainable=True, dtype=tf.float32)
  generated_img_clipped = tf.clip_by_value(generated_img, 0, 255)
  clip_op = generated_img.assign(generated_img_clipped)
  generated_embed_tensor = model(generated_img)
  loss = tf.reduce_sum(tf.square(target_embed_tensor - generated_embed_tensor))
  optimizer = tf.train.AdamOptimizer(learning_rate=lr)
  train_op = optimizer.minimize(loss, var_list=[generated_img])

  sess.run(tf.variables_initializer(optimizer.variables()))
  sess.run(generated_img.assign(create_initial_img(model, target_embed, img_shape, preview)))
  
  real_loss_values = []
  real_loss_values.append(calc_real_loss(sess, model, real_emb, generated_img))
  for i in range(max_steps):
    sess.run(train_op)
    sess.run(clip_op)
    real_loss = calc_real_loss(sess, model, real_emb, generated_img)
    real_loss_values.append(real_loss)
    
#     if i % 20 == 0:      
#       f = plt.figure(figsize=(15, 7))
#       f.add_subplot(1, 3, 1)      
#       plt.imshow(generated_img.eval(sess)[0].astype(np.uint8))      
#       plt.show()
#       print(real_loss)
      
    if real_loss < stop_loss:
      break
      
  return generated_img.eval(sess), real_loss_values

In [18]:
result = []

for i in range(10):
  img_num = 0+i
  img = X[img_num]

  emb, embs, dists = generate_dists(img)
  solved_emb = hack_embs(embs, dists)
  print("distance between real and hacked emb:", np.linalg.norm(solved_emb - emb))

  img_shape = (1, 256, 256, 3)
  generated_img, real_loss_values = generate_reverse_image(
      sess,
      model,
      solved_emb.reshape(1,16).astype(np.float32),
      emb.reshape(1,16).astype(np.float32),
      img_shape,
      cv2.resize(img, dsize=(8, 8), interpolation=cv2.INTER_CUBIC),
      lr=0.1,
      max_steps=300,
      stop_loss=0.05)

  result.append(real_loss_values[-1])
  
  print('Real loss: ', real_loss_values[-1])
  print('Min: %.5f Max: %.5f ' % (np.min(generated_img), np.max(generated_img)))

  f = plt.figure(figsize=(16, 7))
  f.add_subplot(1, 3, 1)
  plt.imshow(img)
  f.add_subplot(1, 3, 2)
  plt.imshow(generated_img[0].astype(np.uint8))
  f.add_subplot(1, 3, 3)
  plt.plot(range(len(real_loss_values)), real_loss_values)
  plt.show()

  
print(result)

Output hidden; open in https://colab.research.google.com to view.